# pYPK0_TDH3_ScCTT1_PGI1

This notebook describes the assembly of the [_Saccharomyces cerevisiae_](http://www.yeastgenome.org)
transcriptional unit vector.

It is made from a linear vector fragment and three PCR products:

- a linearized pTA1 vector for maintenance in _S. cerevisiae_ or _E. coli_ (red dashed line in figure below)
- a promoter PCR product from the pYPKa_Z_TDH3 vector
- a gene PCR product from the pYPKa_A_ScCTT1
- a terminator PCR product from the pYPKa_E_PGI1 vector

The four linear DNA fragments are joined by _in-vivo_ homologous recombination in a
[_Saccharomyces cerevisiae_](http://wiki.yeastgenome.org/index.php/Commonly_used_strains) strain.

![pYPK0_promoter_gene_terminator](tu.png "pYPK0_promoter_gene_terminator")

A part of the [pydna](https://pypi.python.org/pypi/pydna/) package is imported in the code cell below.

In [1]:
from pydna.parsers import parse_primers
from pydna.readers import read
from pydna.amplify import pcr
from pydna.assembly import Assembly

The Yeast Pathway Kit [standard primers](standard_primers.fasta) are read into a dictionary in the code cell below.

In [2]:
p = {x.id: x for x in parse_primers("standard_primers.fasta")}

The backbone vector [pTA1](pTA1.gb) is read from a local file in the code cell below.

In [3]:
backbone = read("pTA1.gb")

The backbone vector is linearized by digestion with [FspAI](https://www.google.com/search?q=FspAI).

In [4]:
from Bio.Restriction import FspAI

In [5]:
linear_backbone = backbone.linearize(FspAI)

The pYPKa derived _E. coli_ plasmids containing 
- [promoter](pYPKa_Z_TDH3.gb)
- [gene](pYPKa_A_ScCTT1.gb)
- [terminator](pYPKa_E_PGI1.gb)

are read into three variables below.

In [6]:
promoter_template   = read("pYPKa_Z_TDH3.gb")
gene_template       = read("pYPKa_A_ScCTT1.gb")
terminator_template = read("pYPKa_E_PGI1.gb")

### PCR
Three DNA fragments are PCR amplified using [standard primers](standard_primers.fasta).

[Suggested PCR programs](#Suggested-PCR-programs) can be found at the end of this document.

In [7]:
prom = pcr(p['577'], p['567'], promoter_template)
gene = pcr(p['468'], p['467'], gene_template)
term = pcr(p['568'], p['578'], terminator_template)

In [8]:
prom.name = "TDH3"
gene.name = "ScCTT1"
term.name = "PGI1"


The fragments will be assembled by _in-vivo_ [homologous recombination](http://www.ncbi.nlm.nih.gov/pubmed/2828185):

In [9]:
asm = Assembly((linear_backbone, prom, gene, term), limit=31)
asm

Assembly
fragments..: 6175bp 930bp 1788bp 1339bp
limit(bp)..: 31
G.nodes....: 8
algorithm..: common_sub_strings

The Assembly object above should normally indicate four fragments and eight nodes.

In [10]:
candidates = asm.assemble_circular()
candidates

[Contig(o9760), Contig(o9760)]

There should normally be two candidates of equal size. These sequences should be identical.

In [11]:
candidate, *rest = candidates

In [12]:
candidate.cseguid() == rest[0].cseguid()

True

In [13]:
candidate.figure()

 -|pTA1_lin|124
|           \/
|           /\
|           124|TDH3|50
|                    \/
|                    /\
|                    50|ScCTT1|37
|                              \/
|                              /\
|                              37|PGI1|242
|                                      \/
|                                      /\
|                                      242-
|                                         |
 -----------------------------------------

The candidate vector is synchronized to the 577 primer. This means that
the plasmid origin is shifted so that it matches the backbone vector.

In [14]:
result = candidate.synced(p['577'])

### Diagnostic PCR confirmation

The structure of the final vector is confirmed by two
separate PCR reactions, one for the promoter and gene and
one for the gene and terminator.

PCR using standard primers 577 and 467 to amplify promoter and gene.

In [15]:
product = pcr( p['577'], p['467'], result)

A correct clone should give this size in base pairs:

In [16]:
print(len(product))

2668


If the promoter is missing from the assembly, the PCR product will have this size in base pairs:

In [17]:
print(len(product) - len(prom))

1738


If the gene is missing from the assembly, the PCR product will have this size in base pairs:

In [18]:
print(len(product) - len(gene))

880


PCR using standard primers 468 and 578 to amplify gene and terminator.

In [19]:
product2 = pcr( p['468'], p['578'], result)

A correct clone should give this size:

In [20]:
print(len(product2))

3090


If the gene is missing from the assembly, the PCR product will have this size in base pairs:

In [21]:
print(len(product2) - len(gene))

1302


If the terminator is missing from the assembly, the PCR product will have this size in base pairs:

In [22]:
print(len(product2) - len(term))

1751


The cseguid checksum for the resulting plasmid is calculated for future reference.
The [cseguid checksum](http://pydna.readthedocs.org/en/latest/pydna.html#pydna.utils.cseguid)
uniquely identifies a circular double stranded sequence.

In [23]:
print(result.cseguid())

zaY85FDrm6zqHlFarxifz-S3DEU


The file name is based on the promoter, gene and terminator designations.

In [24]:
result.locus = "TU:ScCTT1"[:16]
result.definition = "pYPK0_TDH3_ScCTT1_PGI1"

Sequence is stamped with cseguid checksum. This can be used to verify the
integrity of the sequence file.

In [25]:
result.stamp("cSEGUID")

zaY85FDrm6zqHlFarxifz-S3DEU

Write sequence to a local file.

In [26]:
result.write(f"{result.definition}.gb")

pYPK0_TDH3_ScCTT1_PGI1.gb

### Suggested PCR programs

For the [amplification](#PCR) of promoter, gene and terminator.

In [27]:
print(prom.forward_primer.name, prom.reverse_primer.name)
print(prom.program())

577 567
|95°C|95°C               |    |tmf:64.6
|____|_____          72°C|72°C|tmr:69.7
|3min|30s  \ 57.1°C _____|____|45s/kb
|    |      \______/ 0:41|5min|GC 38%
|    |       30s         |    |930bp


In [28]:
print(gene.forward_primer.name, gene.reverse_primer.name)
print(gene.program())

468 467
|95°C|95°C               |    |tmf:76.9
|____|_____          72°C|72°C|tmr:67.9
|3min|30s  \ 59.1°C _____|____|45s/kb
|    |      \______/ 1:20|5min|GC 41%
|    |       30s         |    |1788bp


In [29]:
print(term.forward_primer.name, term.reverse_primer.name)
print(term.program())

568 578
|95°C|95°C               |    |tmf:66.1
|____|_____          72°C|72°C|tmr:65.0
|3min|30s  \ 57.6°C _____|____|45s/kb
|    |      \______/ 1:00|5min|GC 39%
|    |       30s         |    |1339bp
